In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/brandonmanley/Desktop/PhD/oam_pheno/dijet_dsa')
import dsa_mc.dijet as dijet
import random
from IPython.display import display, Math

# setup plotting
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 12
plt.rcParams["legend.fontsize"] = 12
plt.rcParams["axes.labelsize"] = 14  
plt.rcParams["xtick.labelsize"] = 12  
plt.rcParams["ytick.labelsize"] = 12
plt.rcParams["axes.titlesize"] = 16  
plt.rcParams["text.usetex"] = True
plt.rcParams["text.latex.preamble"] = r"\usepackage{amsmath}"
%config InlineBackend.figure_format = 'retina'

In [5]:
# find moment params that satisfy \int^0.1_10^-5 (OAM) dx < 0.5

def get_param_guess():
    ps = {}
    for amp in ['I3u', 'I3d', 'I3s', 'I3T', 'I4', 'I5']:
        ps[amp] = {}
        for basis in ['eta', 's10', '1']:
            ps[amp][basis] = np.random.uniform(-10, 10)
    return ps

Q2 = 10

djs = {}
djs['dis+sidis'] = dijet.DIJET(fit_type='dis')
djs['+pp'] = dijet.DIJET(fit_type='pp')

mparams = {}
mparams['dis+sidis'] = []
mparams['+pp'] = []

for irep in range(1, 398):
    for fit in ['+pp']:
        print(irep)
        
        found_params = False
        while not found_params:
            guess = get_param_guess()
            djs[fit].set_temp_params(guess, irep)

            safe_replica = True
            for xmax in [10**(-4), 10**(-3), 10**(-2), 10**(-1)]:
                integrated_oam_value = djs[fit].get_IntegratedPDF('oam', Q2, xmax=xmax)
                if np.abs(integrated_oam_value) > 1.0: safe_replica = False

            if safe_replica: found_params = True
                
        mparams[fit].append(guess)


--> loaded unpol. amp. data from /dipoles/narr_ymin4.61_ymax14.91_AAMS09.dat
--> loaded pol. amp. data from /dipoles/d05-rc/
--> loaded params from /dipoles/replica_params_dis.csv
--> loaded random moment params from /dipoles/moment_params_dis.csv
--> loaded replica 1
--> loaded unpol. amp. data from /dipoles/narr_ymin4.61_ymax14.91_AAMS09.dat
--> loaded pol. amp. data from /dipoles/d05-rc/
--> loaded params from /dipoles/replica_params_pp_2.csv
--> loaded random moment params from /dipoles/moment_params_pp.csv
--> loaded replica 1
1
2
3
4
5
6
7


KeyboardInterrupt: 

In [7]:
len(mparams['dis+sidis'])

101

In [9]:
# reformat and save parameters
par_arrs = {}
par_arrs['dis+sidis'] = []
par_arrs['+pp'] = []

header = ['nrep'] + [f'{amp}{basis}' for amp in ['I3u', 'I3d', 'I3s', 'I3T', 'I4', 'I5'] for basis in ['eta', 's10', '1']]
header_str = ",".join(header)

for irep in range(len(mparams['dis+sidis'])):
    for fit in ['dis+sidis', '+pp']:
        # if irep == 0: 
        #     par_arrs[fit].append(header)
        
        if fit == '+pp' and irep > 397: continue
        # print(irep)
        params = []
        for amp in ['I3u', 'I3d', 'I3s', 'I3T', 'I4', 'I5']:
            for basis in ['eta', 's10', '1']:
                params.append(mparams[fit][irep][amp][basis])
        par_arrs[fit].append([irep+1] + params)


np.savetxt('moment_params_dis.csv', par_arrs['dis+sidis'], delimiter=",", header=header_str, comments='')
np.savetxt('moment_params_pp.csv', par_arrs['+pp'], delimiter=",", header=header_str, comments='')

In [15]:
import pandas as pd
df = pd.read_csv('moment_params_dis.csv')


      nrep    I3ueta    I3us10      I3u1    I3deta    I3ds10      I3d1  \
0      1.0  2.523849  2.572098  2.746114 -8.270481  5.173837  6.922209   
1      2.0  2.152670  9.996407  6.106308  8.503620  4.409372  7.385422   
2      3.0 -4.432514 -6.132048  7.568362  9.266310 -7.349874 -2.096130   
3      4.0 -7.247517  0.674610 -8.342382  5.495344  2.144427 -4.128619   
4      5.0 -1.434481 -5.337515  8.554989  4.815798 -8.768785  4.975701   
..     ...       ...       ...       ...       ...       ...       ...   
96    97.0  2.557494  2.334304 -6.454207  9.899939  2.102750  9.978124   
97    98.0  1.730617  0.178111 -5.831335  1.033545 -1.549116  8.105624   
98    99.0  1.785892  9.587630  1.662111 -1.696511  9.893021 -0.035573   
99   100.0 -5.353304  6.651543  1.384365  2.176106 -6.227649  6.114506   
100  101.0 -8.133833 -1.347383  7.600548  2.529769  6.492208  2.055193   

       I3seta    I3ss10      I3s1    I3Teta    I3Ts10      I3T1     I4eta  \
0   -8.981557 -0.356087  9.140989 